In [2]:
import sys
import cv2
import mediapipe as mp
import glob
import numpy as np
import json
import os
import time
from os.path import exists

In [3]:
# Data paths
json_file = '/mnt/HDD04/WASL Dataset/WLASL-master/start_kit/WLASL_v0.3.json'

with open(json_file) as ipf:
    content = json.load(ipf)

datapath = '/mnt/HDD04/WASL Dataset/WLASL-master/start_kit/videos/'
vid_savepath = '/mnt/HDD04/WASL Dataset/Scripts/annotated_videos/'
annot_path = '/mnt/HDD04/WASL Dataset/Scripts/annotated_images/'
skelpath = '/mnt/HDD04/WASL Dataset/Scripts/skeletons/'
frame_path = skelpath.replace('skeletons', 'images')
json_path = skelpath.replace('skeletons', 'jsons')

In [4]:
files = glob.glob(datapath + '*mp4')
print('Num files present:', len(files))
print('Num files should be:', sum([len(c['instances']) for c in content]))
print(files[0])

Num files present: 3930
Num files should be: 21083
/mnt/HDD04/WASL Dataset/WLASL-master/start_kit/videos/23865.mp4


In [5]:
print('Class:', content[0]['gloss'])
print('sample:', content[0]['instances'][0])

Class: book
sample: {'bbox': [385, 37, 885, 720], 'fps': 25, 'frame_end': -1, 'frame_start': 1, 'instance_id': 0, 'signer_id': 118, 'source': 'aslbrick', 'split': 'train', 'url': 'http://aslbricks.org/New/ASL-Videos/book.mp4', 'variation_id': 0, 'video_id': '69241'}


In [6]:
content[0]['instances'][0]['video_id']

'69241'

In [7]:
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_hands = mp.solutions.hands

In [19]:
# extract skeleton
start_time = time.time()
cnt = 0
for i, c in enumerate(content):
    for j in range(len(c['instances'])):
        gloss = c['gloss']  # i.e., class 
        fname = datapath + c['instances'][j]['video_id'] + '.mp4'
        
        if exists(fname):
            cnt += 1
            now = time.time()
            print('Processing content ' + str(i+1) + '/' + str(len(content)) +
                  ', instance ' + str(j+1) + '/' + str(len(c['instances'])))
            cap = cv2.VideoCapture(fname)
            fps = cap.get(cv2.CAP_PROP_FPS)
#             start_frame = c['instances'][j]['frame_start'] - 1
            start_frame = 0
            end_frame = int(cap.get(cv2.CAP_PROP_FRAME_COUNT)) - 1
#             if c['instances'][j]['frame_end'] != -1:
#                 end_frame = c['instances'][j]['frame_end']
            subfolder_frame = frame_path + fname.split('/')[-1][:-4]
            if not exists(subfolder_frame):
                os.mkdir(subfolder_frame)
                bbox = c['instances'][j]['bbox']
                # extract frames
    #             for frame_no in range(start_frame, end_frame + 1):
                for frame_no in range(start_frame, end_frame + 1):
                    cap.set(1,frame_no)  # Where frame_no is the frame you want
                    ret, frame = cap.read()  # Read the frame
#                     frame = frame[bbox[1]:bbox[3], bbox[0]:bbox[2]]
                    cv2.imwrite(subfolder_frame + '/' + fname.split('/')[-1][:-4] + '_frame' + f'{frame_no:05d}' + '.jpg', frame)
            
            # MediaPipe
            subfolder_skel = skelpath + fname.split('/')[-1][:-4]
            if not exists(subfolder_skel):
                os.mkdir(subfolder_skel)
                IMAGE_FILES = sorted(glob.glob(subfolder_frame + '/*jpg'))
                with mp_hands.Hands(
                    static_image_mode=True,
                    max_num_hands=2,
                    min_detection_confidence=0.5) as hands:

                    image = cv2.flip(cv2.imread(IMAGE_FILES[0]), 1)
                    fname2 = IMAGE_FILES[0].split('/')[-1]
                    prev_results = hands.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))

                    for idx, file in enumerate(IMAGE_FILES):
                        # Read an image, flip it around y-axis for correct handedness output (see
                        # above).
                        image = cv2.flip(cv2.imread(file), 1)
                        fname2 = file.split('/')[-1]
                        # Convert the BGR image to RGB before processing.
                        results = hands.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))

                        # Print handedness and draw hand landmarks on the image.
    #                     print('Frame: ', idx, '/', len(IMAGE_FILES))
                        # print('Handedness:', results.multi_handedness)
                        if not results.multi_hand_landmarks:
                            if not prev_results.multi_hand_landmarks:
                                continue
                            else:
                                results = prev_results  # to prevent frame elimination due to non-existent hands
                        prev_results = results
                        image_height, image_width, _ = image.shape
                        annotated_image = image.copy()
                        data = []

                        for hand_id, hand_landmarks in enumerate(results.multi_hand_landmarks):


                          # print('hand_landmarks:', hand_landmarks)
                          # print(
                          #     f'Index finger tip coordinates: (',
                          #     f'{hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP].x * image_width}, '
                          #     f'{hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP].y * image_height})'
                          # )
                          # print(dir(hand_landmarks.landmark[0]))
                          # print(hand_landmarks.landmark[0])
                          # print(hand_landmarks.landmark[0].x)

                            for land_id, mark in enumerate(hand_landmarks.landmark):

                                if land_id == 0:
                                    x_temp = [mark.x]
                                    y_temp = [mark.y]
                                    z_temp = [mark.z]
                                else:
                                    x_temp = np.concatenate((x_temp, [mark.x]), 0)
                                    y_temp = np.concatenate((y_temp, [mark.y]), 0)
                                    z_temp = np.concatenate((z_temp, [mark.z]), 0)

                            data.append(x_temp)
                            data.append(y_temp)
                            data.append(z_temp)
                            mp_drawing.draw_landmarks(
                                  annotated_image,
                                  hand_landmarks,
                                  mp_hands.HAND_CONNECTIONS,
                                  mp_drawing_styles.get_default_hand_landmarks_style(),
                                  mp_drawing_styles.get_default_hand_connections_style())
                        

                        with open(subfolder_skel + '/' + fname2.replace('jpg', 'txt'), 'w') as f:
                            for elem in data:
                                for e in elem:
                                    f.write(str(e) + ' ')
                                f.write('\n')
                        subfolder_im = annot_path + fname.split('/')[-1][:-4]
                        if not exists(subfolder_im):
                            os.mkdir(subfolder_im)
                        cv2.imwrite(subfolder_im + '/' + fname2.replace('jpg', 'png'), cv2.flip(annotated_image, 1))
                    
                    # create video
#                     files = sorted(glob.glob(subfolder_im + '/*png'))
#                     frame = cv2.imread(files[0])
#                     height, width, layers = frame.shape
#                     video_name = vid_savepath + fname.split('/')[-1][:-4] + '.avi'
#                     video = cv2.VideoWriter(video_name, 0, fps, (width, height))
#                     for image in files:
#                         video.write(cv2.imread(image))
#                     video.release()
                    
            print('Elapsed time:', time.time() - now, 'sec')
            print('Num. processed file:' + str(cnt) + '/' + str(len(files)))
            print('Total Elapsed time:', (time.time() - start_time)/3600, 'hrs')
            
            # save content as json
            json_name = json_path + fname.split('/')[-1][:-3] + 'json'
            
            if not exists(json_name):
                content2 = c['instances'][j]
                content2['gloss'] = c['gloss']
                json_object = json.dumps(content2, indent=4)
                # Writing to sample.json
                with open(json_name, "w") as outfile:
                    outfile.write(json_object)

#             if j == 4:
#                 break
#             break
        
#     break
    

Processing content 1/2000, instance 3/40
Elapsed time: 9.211495876312256 sec
Num. processed file:1/3930
Total Elapsed time: 0.002559165557225545 hrs
Processing content 1/2000, instance 5/40
Elapsed time: 20.832849740982056 sec
Num. processed file:2/3930
Total Elapsed time: 0.00834697491592831 hrs
Processing content 1/2000, instance 6/40
Elapsed time: 7.164451360702515 sec
Num. processed file:3/3930
Total Elapsed time: 0.010337637530432807 hrs
Processing content 1/2000, instance 7/40
Elapsed time: 13.826125860214233 sec
Num. processed file:4/3930
Total Elapsed time: 0.014178891513082716 hrs
Processing content 1/2000, instance 31/40
Elapsed time: 22.60040044784546 sec
Num. processed file:5/3930
Total Elapsed time: 0.02045739902390374 hrs
Processing content 1/2000, instance 32/40
Elapsed time: 32.55447578430176 sec
Num. processed file:6/3930
Total Elapsed time: 0.029501130779584248 hrs
Processing content 2/2000, instance 3/35
Elapsed time: 14.074846982955933 sec
Num. processed file:7/3930

Elapsed time: 18.131802082061768 sec
Num. processed file:56/3930
Total Elapsed time: 0.2771898729933633 hrs
Processing content 11/2000, instance 1/23
Elapsed time: 5.020173072814941 sec
Num. processed file:57/3930
Total Elapsed time: 0.27858531574408213 hrs
Processing content 11/2000, instance 2/23
Elapsed time: 8.989372491836548 sec
Num. processed file:58/3930
Total Elapsed time: 0.2810828635427687 hrs
Processing content 11/2000, instance 4/23
Elapsed time: 20.578941345214844 sec
Num. processed file:59/3930
Total Elapsed time: 0.2868000345759922 hrs
Processing content 11/2000, instance 5/23
Elapsed time: 19.23899006843567 sec
Num. processed file:60/3930
Total Elapsed time: 0.292144906255934 hrs
Processing content 11/2000, instance 6/23
Elapsed time: 25.074032068252563 sec
Num. processed file:61/3930
Total Elapsed time: 0.29911118288834887 hrs
Processing content 11/2000, instance 21/23
Elapsed time: 25.509172677993774 sec
Num. processed file:62/3930
Total Elapsed time: 0.30619763990243

Elapsed time: 13.56486177444458 sec
Num. processed file:111/3930
Total Elapsed time: 0.555443209277259 hrs
Processing content 20/2000, instance 18/21
Elapsed time: 10.756657838821411 sec
Num. processed file:112/3930
Total Elapsed time: 0.5584317822588815 hrs
Processing content 20/2000, instance 19/21
Elapsed time: 24.159830808639526 sec
Num. processed file:113/3930
Total Elapsed time: 0.5651437985897064 hrs
Processing content 21/2000, instance 3/21
Elapsed time: 12.475266218185425 sec
Num. processed file:114/3930
Total Elapsed time: 0.5686096310615539 hrs
Processing content 21/2000, instance 5/21
Elapsed time: 20.799869537353516 sec
Num. processed file:115/3930
Total Elapsed time: 0.5743881914350721 hrs
Processing content 22/2000, instance 1/21
Elapsed time: 7.316747665405273 sec
Num. processed file:116/3930
Total Elapsed time: 0.5764212535487281 hrs
Processing content 22/2000, instance 2/21
Elapsed time: 11.10705018043518 sec
Num. processed file:117/3930
Total Elapsed time: 0.57950719

Elapsed time: 17.584959983825684 sec
Num. processed file:166/3930
Total Elapsed time: 0.8204641904433568 hrs
Processing content 29/2000, instance 1/21
Elapsed time: 6.728756904602051 sec
Num. processed file:167/3930
Total Elapsed time: 0.8223337050278982 hrs
Processing content 29/2000, instance 2/21
Elapsed time: 6.7069127559661865 sec
Num. processed file:168/3930
Total Elapsed time: 0.8241973010036681 hrs
Processing content 29/2000, instance 17/21
Elapsed time: 29.452685356140137 sec
Num. processed file:169/3930
Total Elapsed time: 0.8323793627156152 hrs
Processing content 29/2000, instance 18/21
Elapsed time: 21.87047553062439 sec
Num. processed file:170/3930
Total Elapsed time: 0.8384555758370293 hrs
Processing content 30/2000, instance 2/21
Elapsed time: 20.67864489555359 sec
Num. processed file:171/3930
Total Elapsed time: 0.8442002534866333 hrs
Processing content 30/2000, instance 20/21
Elapsed time: 15.6269850730896 sec
Num. processed file:172/3930
Total Elapsed time: 0.84854228

Elapsed time: 22.21477723121643 sec
Num. processed file:221/3930
Total Elapsed time: 1.092688848376274 hrs
Processing content 41/2000, instance 1/20
Elapsed time: 5.750569581985474 sec
Num. processed file:222/3930
Total Elapsed time: 1.0942867779069476 hrs
Processing content 41/2000, instance 4/20
Elapsed time: 9.869845390319824 sec
Num. processed file:223/3930
Total Elapsed time: 1.0970289572079976 hrs
Processing content 41/2000, instance 16/20
Elapsed time: 21.137044429779053 sec
Num. processed file:224/3930
Total Elapsed time: 1.1029008493820827 hrs
Processing content 41/2000, instance 17/20
Elapsed time: 17.372077226638794 sec
Num. processed file:225/3930
Total Elapsed time: 1.107727071377966 hrs
Processing content 41/2000, instance 18/20
Elapsed time: 21.03780484199524 sec
Num. processed file:226/3930
Total Elapsed time: 1.1135716071393755 hrs
Processing content 41/2000, instance 19/20
Elapsed time: 28.97780179977417 sec
Num. processed file:227/3930
Total Elapsed time: 1.121621414

...

Elapsed time: 7.634916067123413 sec
Num. processed file:3712/3930
Total Elapsed time: 15.061497816112306 hrs
Processing content 1801/2000, instance 1/7
Elapsed time: 11.781042337417603 sec
Num. processed file:3713/3930
Total Elapsed time: 15.064771339694659 hrs
Processing content 1801/2000, instance 6/7
Elapsed time: 23.210297346115112 sec
Num. processed file:3714/3930
Total Elapsed time: 15.071219963961177 hrs
Processing content 1802/2000, instance 1/7
Elapsed time: 8.472937107086182 sec
Num. processed file:3715/3930
Total Elapsed time: 15.073574172258377 hrs
Processing content 1803/2000, instance 1/7
Elapsed time: 6.569746494293213 sec
Num. processed file:3716/3930
Total Elapsed time: 15.075399868753221 hrs
Processing content 1804/2000, instance 2/7
Elapsed time: 20.64231586456299 sec
Num. processed file:3717/3930
Total Elapsed time: 15.081135210394859 hrs
Processing content 1805/2000, instance 1/7
Elapsed time: 4.771261930465698 sec
Num. processed file:3718/3930
Total Elapsed time: 

Elapsed time: 8.346247673034668 sec
Num. processed file:3766/3930
Total Elapsed time: 15.245824065738255 hrs
Processing content 1847/2000, instance 1/7
Elapsed time: 8.121058702468872 sec
Num. processed file:3767/3930
Total Elapsed time: 15.248080673019091 hrs
Processing content 1847/2000, instance 5/7
Elapsed time: 19.51873016357422 sec
Num. processed file:3768/3930
Total Elapsed time: 15.253503117892476 hrs
Processing content 1848/2000, instance 1/7
Elapsed time: 3.7744860649108887 sec
Num. processed file:3769/3930
Total Elapsed time: 15.254552373025152 hrs
Processing content 1848/2000, instance 7/7
Elapsed time: 13.455096006393433 sec
Num. processed file:3770/3930
Total Elapsed time: 15.2582904646794 hrs
Processing content 1849/2000, instance 1/7
Elapsed time: 9.166566371917725 sec
Num. processed file:3771/3930
Total Elapsed time: 15.26083743247721 hrs
Processing content 1850/2000, instance 1/7
Elapsed time: 5.125223875045776 sec
Num. processed file:3772/3930
Total Elapsed time: 15.

Elapsed time: 7.573947429656982 sec
Num. processed file:3820/3930
Total Elapsed time: 15.400555051101579 hrs
Processing content 1899/2000, instance 1/7
Elapsed time: 6.896258592605591 sec
Num. processed file:3821/3930
Total Elapsed time: 15.402471361292733 hrs
Processing content 1900/2000, instance 1/7
Elapsed time: 4.902221918106079 sec
Num. processed file:3822/3930
Total Elapsed time: 15.403834059039752 hrs
Processing content 1902/2000, instance 1/7
Elapsed time: 6.046755075454712 sec
Num. processed file:3823/3930
Total Elapsed time: 15.405514621271028 hrs
Processing content 1903/2000, instance 1/7
Elapsed time: 7.373892307281494 sec
Num. processed file:3824/3930
Total Elapsed time: 15.407563301722208 hrs
Processing content 1904/2000, instance 6/7
Elapsed time: 23.93520188331604 sec
Num. processed file:3825/3930
Total Elapsed time: 15.414212923844655 hrs
Processing content 1906/2000, instance 1/7
Elapsed time: 6.732337236404419 sec
Num. processed file:3826/3930
Total Elapsed time: 15

Elapsed time: 19.621591329574585 sec
Num. processed file:3874/3930
Total Elapsed time: 15.569848133126895 hrs
Processing content 1950/2000, instance 1/7
Elapsed time: 6.793457746505737 sec
Num. processed file:3875/3930
Total Elapsed time: 15.571735906865861 hrs
Processing content 1950/2000, instance 5/7
Elapsed time: 23.93531060218811 sec
Num. processed file:3876/3930
Total Elapsed time: 15.578385247124565 hrs
Processing content 1951/2000, instance 1/7
Elapsed time: 13.485190153121948 sec
Num. processed file:3877/3930
Total Elapsed time: 15.58213202436765 hrs
Processing content 1952/2000, instance 1/7
Elapsed time: 10.790446043014526 sec
Num. processed file:3878/3930
Total Elapsed time: 15.585129925211271 hrs
Processing content 1953/2000, instance 1/7
Elapsed time: 3.792362689971924 sec
Num. processed file:3879/3930
Total Elapsed time: 15.586184024148517 hrs
Processing content 1953/2000, instance 6/7
Elapsed time: 15.620715141296387 sec
Num. processed file:3880/3930
Total Elapsed time:

Elapsed time: 7.6982526779174805 sec
Num. processed file:3928/3930
Total Elapsed time: 15.722455739643838 hrs
Processing content 1998/2000, instance 2/7
Elapsed time: 13.237221956253052 sec
Num. processed file:3929/3930
Total Elapsed time: 15.726133689946598 hrs
Processing content 2000/2000, instance 1/7
Elapsed time: 6.202758550643921 sec
Num. processed file:3930/3930
Total Elapsed time: 15.727857748733626 hrs


In [13]:
with open(json_name) as ipf:
    temp_content = json.load(ipf)

In [14]:
temp_content

{'bbox': [95, 0, 1180, 720],
 'fps': 25,
 'frame_end': -1,
 'frame_start': 1,
 'instance_id': 4,
 'signer_id': 109,
 'source': 'valencia-asl',
 'split': 'train',
 'url': 'https://www.youtube.com/watch?v=aGtIHKEdCds',
 'variation_id': 0,
 'video_id': '68012',
 'gloss': 'book'}

In [21]:
from collections import Counter

[print(sum()) for c in content]
# Counter()

40
35
30
26
26
26
25
25
24
23
23
22
22
22
22
22
22
22
21
21
21
21
21
21
21
21
21
21
21
21
21
21
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
19
19
19
19
19
19
19
19
19
19
19
19
19
19
19
19
19
19
19
19
19
19
19
19
19
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
16
16
16
16
16
16
16
16
16
16
16
16
16
15
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
1

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,

In [23]:
len(content[0]['instances'])

40